In [1]:
import re
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
start_time = time.time()

In [4]:
with open("/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/bicorpus", "r") as infile:
    corpus = infile.readlines()

In [5]:
corpus[0:2]

['\ufeffAmit csak megtudhattam a szegény Werther történetéről, szorgalmasan összegyűjtöttem, és most elétek bocsátom, és tudom, hálásak lesztek érte.\tTất cả những gì tôi có thể tìm được từ câu chuyện của chàng Werther tội nghiệp, tôi đã chuyên cần thu góp lại và đem trình với các bạn dưới đây, và tôi biết rằng các bạn sẽ cảm ơn tôi.\n',
 'És mégis - csakugyan ártatlan vagyok?\tThế nhưng - đã chắc đâu tôi hoàn toàn vô tội?\n']

In [6]:
len(corpus)

143451

In [7]:
bicorpus = {}
for line in corpus[50000:]:
    line = line.strip()
    src, tgt = line.split("\t")
    if (src in bicorpus):
        bicorpus[src] += [tgt]
    else:
        bicorpus[src] = [tgt]

In [8]:
len(bicorpus)

91160

In [9]:
with open("/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/sorted_gold_dict_0.5", "r") as infile:
    d = infile.readlines()

In [10]:
dictionary = {}
#pathScoreDict = {}
for line in d:
    line = line.strip()
    src, tgt, path, score = line.split("\t")
    if (src in dictionary):
        dictionary[src] += [tgt]
    else:
        dictionary[src] = [tgt]
    #key = src + "\t" + tgt
    #pathScoreDict[key] = {"path":path, "score":score, "count":0} 

In [11]:
len(dictionary)

18917

In [12]:
dictionary["fölé"]

['trên', 'bên trên', 'qua', 'trong', 'lên']

In [22]:
import json
with open("/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/pathScoreDict.json", "r") as infile:
    pathScoreDict = json.load(infile)

In [14]:
len(pathScoreDict)

57052

In [15]:
def find_word(word, sent):
    search_string = r"\b" + word + r"\b"
    search_results = re.search(search_string, sent, flags=re.IGNORECASE)
    if search_results is None:
        return False
    else:
        return True

In [16]:
line = "bên nhiều trênfölé"

In [17]:
find_word("nhiều", line)

True

In [18]:
outfile = open("/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/confirmLocationGoldDict.txt", "a")
for word in dictionary.keys():
    for src_sent in bicorpus.keys():
        try:
            if find_word(word, src_sent):
              translations = dictionary[word]
              tgt_sents = bicorpus[src_sent]
              for translation in translations:
                  for tgt_sent in tgt_sents:
                      if find_word(translation, tgt_sent):
                          outfile.write(word + "\t" + translation + "\t" + src_sent + "\t" + tgt_sent + "\n") 
                          key = word + "\t" + translation
                          pathScoreDict[key]["count"] += 1 
        except:
            continue
outfile.close()


In [19]:
with open("/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/pathScoreDict.json", "w") as outfile:
    json.dump(pathScoreDict, outfile)

In [23]:
outfile = open("/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/confirmedGoldDict.txt", "w")
for key in pathScoreDict.keys():
    item = pathScoreDict[key]
    if (item["count"] > 0):
      outfile.write(key + "\t" + str(item["count"]) + "\t" + item["path"] + "\t" + item["score"] + "\n")
outfile.close()

In [21]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 5786.107335567474 seconds ---
